In [ ]:
import numpy as np
import pickle
from pathlib import Path

from IPython.display import display
from lib.util_plot import *
from lib.constants import *
from lib.multilevel_analysis import *
from lib.pandas_impl import *
from lib.pandas_stats_VIS import *

from lib.matrix_analyzer import *

import lib.H01_preprocessing.h01_constants as H01

### Load and prepare data

In [ ]:
data_folder = Path.cwd() / 'data' / 'H01'
eval_folder = Path.cwd() / 'data' / 'eval' / 'H01_24-12-17'
eval_folder.mkdir(parents=True, exist_ok=True)
plot_folder = eval_folder / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
filename = eval_folder / "multilevel_analysis.pkl"
with open(filename, 'rb') as file:
    multilevel_analysis = pickle.load(file)

stats = multilevel_analysis.stats
df_summary = multilevel_analysis.df_summary

select neuron IDs for cellular-level matrix plots

In [ ]:
pre_ids = set(df_summary.index.get_level_values("pre_id_mapped"))
post_ids = set(df_summary.index.get_level_values("post_id_mapped"))
all_ids = pre_ids.union(post_ids)
all_ids.remove(-1)   
no_presynaptic = all_ids - pre_ids

neuron_domain_pre = get_neuron_to_neuron_domain(df_summary, "pre_celltype_merged", "post_celltype_merged", \
                                                 celltype_order= H01.UNK_EXC_INH_OTHER, ignored_neuron_ids=[-1])
neuron_domain_post = get_neuron_to_neuron_domain(df_summary, "pre_celltype_merged", "post_celltype_merged", \
                                                  celltype_order=H01.UNK_EXC_INH_OTHER, ignored_neuron_ids=[-1])

In [ ]:
initPlotSettings(False)

COLORS_CATEGORICAL_MUTED = sns.color_palette("muted") 
COLORS_CATEGORICAL_DARK = sns.color_palette("dark") 
COLORS_CATEGORICAL_COLORBLIND = sns.color_palette("colorblind") 
COLORS_CATEGORICAL_BRIGHT = sns.color_palette("bright") 
COLORS_CATEGORICAL_PASTEL = sns.color_palette("pastel") 

C_EMPIRICAL = COLORS_CATEGORICAL[2]
C_H0 = COLOR_EMPIRICAL
C_M1 = COLORS_CATEGORICAL_MUTED[7]
C_M2 = COLORS_CATEGORICAL_BRIGHT[7]
C_M3 = COLORS_CATEGORICAL_PASTEL[7]

### Aggregate statistics and preference matrices

#### Synapse counts by celltype

In [ ]:
datasets = [stats.to_numpy(SELECTION_CELLTYPE, EMPIRICAL), 
            stats.to_numpy(SELECTION_CELLTYPE, MODEL_NULL),
            stats.to_numpy(SELECTION_CELLTYPE, MODEL_P)]

img = plot_bar_chart(datasets,
                     #dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P], 
                     colors=[C_EMPIRICAL, C_H0, C_M1],
                     x_labels=SELECTION_CELLTYPE,
                     y_lim=(10, 10**5), use_log=True, error_bars=True,
                     adjust_left=0.2, fig_size=figsize_mm_to_inch(50,40), 
                     filename=plot_folder/f"H01_{SYNCOUNT_POPULATION}.png")
display(img)

In [ ]:
datasets = [stats.to_numpy(SELECTION_CELLTYPE_H01_ALL, EMPIRICAL), 
            stats.to_numpy(SELECTION_CELLTYPE_H01_ALL, MODEL_NULL),
            stats.to_numpy(SELECTION_CELLTYPE_H01_ALL, MODEL_P)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P], 
                     colors=[C_EMPIRICAL, C_H0, C_M1],
                     x_labels=SELECTION_CELLTYPE_H01_ALL,
                     y_lim=(10, 10**8), use_log=True, error_bars=True,
                     adjust_left=0.08, fig_size=figsize_mm_to_inch(150,50), 
                     filename=plot_folder/f"H01_{SYNCOUNT_POPULATION}_all.png")
display(img)

In [ ]:
matrix_analyzer = ConnectomeMatrixAnalyzer(df_summary, plot_folder)
matrix_analyzer.set_selection(exclude_filter={})
matrix_analyzer.set_data_columns("model-P_preference")
matrix_analyzer.build_matrix(["pre_celltype_merged"], 
                             ["post_celltype_merged"],    
                             row_domains = H01.UNK_EXC_INH_OTHER[1:3],  
                             col_domains = H01.UNK_EXC_INH_OTHER[1:3],
                             value_label_map = {
                                 "pre_celltype_merged" : H01.CELLTYPE_LABELS,
                                 "post_celltype_merged" : H01.CELLTYPE_LABELS,
                             },
                             aggregation_fn="mean",
                             default_value=1)
matrix_analyzer.render_matrix("H01-model-P-preference", 
                              colormap_name="coolwarm", vmin=-1, vmax=1, normalization_function=NormalizePreferenceValue(0.5, 4),
                              col_separator_lines=True, row_separator_lines=True, high_res=False)

In [ ]:
matrix_analyzer = ConnectomeMatrixAnalyzer(df_summary, plot_folder)
matrix_analyzer.set_selection(exclude_filter={})
matrix_analyzer.set_data_columns("model-P_preference")
matrix_analyzer.build_matrix(["pre_celltype_merged"], 
                             ["post_celltype_merged"],    
                             row_domains = H01.EXC_INH_OTHER_UNK,  
                             col_domains = H01.EXC_INH_OTHER_UNK,
                             value_label_map = {
                                 "pre_celltype_merged" : H01.CELLTYPE_LABELS,
                                 "post_celltype_merged" : H01.CELLTYPE_LABELS,
                             },
                             aggregation_fn="mean",
                             default_value=1)
matrix_analyzer.render_matrix("H01-model-P-preference-all", 
                              colormap_name="coolwarm", vmin=-1, vmax=1, normalization_function=NormalizePreferenceValue(0.5, 4),
                              col_separator_lines=True, row_separator_lines=True, high_res=False)

#### Synapse counts by celltype/compartment

In [ ]:
datasets = [stats.to_numpy(SELECTION_EXC_SUBCELLULAR, EMPIRICAL), 
            stats.to_numpy(SELECTION_EXC_SUBCELLULAR, MODEL_NULL),
            stats.to_numpy(SELECTION_EXC_SUBCELLULAR, MODEL_P),
            stats.to_numpy(SELECTION_EXC_SUBCELLULAR, MODEL_PS)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P, STR_PS], 
                     colors=[C_EMPIRICAL, C_H0, C_M1, C_M2],
                     x_labels=SELECTION_EXC_SUBCELLULAR,
                     y_lim=(1, 10**5), use_log=True, error_bars=True,
                     adjust_left=0.1, fig_size=figsize_mm_to_inch(100,50), 
                     filename=plot_folder/f"H01_{SYNCOUNT_EXC_SUBCELLULAR}.png")
display(img)

In [ ]:
datasets = [stats.to_numpy(SELECTION_INH_SUBCELLULAR[:-1], EMPIRICAL), 
            stats.to_numpy(SELECTION_INH_SUBCELLULAR[:-1], MODEL_NULL),
            stats.to_numpy(SELECTION_INH_SUBCELLULAR[:-1], MODEL_P),
            stats.to_numpy(SELECTION_INH_SUBCELLULAR[:-1], MODEL_PS)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P, STR_PS], 
                     colors=[C_EMPIRICAL, C_H0, C_M1, C_M2],
                     x_labels=SELECTION_INH_SUBCELLULAR[:-1],
                     y_lim=(1, 10**5), use_log=True, error_bars=True,
                     adjust_left=0.15, fig_size=figsize_mm_to_inch(70,40), 
                     filename=plot_folder/f"H01_{SYNCOUNT_INH_SUBCELLULAR}.png")
display(img)

In [ ]:
datasets = [stats.to_numpy(SELECTION_EXC_COMPARTMENT_H01_ALL, EMPIRICAL), 
            stats.to_numpy(SELECTION_EXC_COMPARTMENT_H01_ALL, MODEL_NULL),
            stats.to_numpy(SELECTION_EXC_COMPARTMENT_H01_ALL, MODEL_P),
            stats.to_numpy(SELECTION_EXC_COMPARTMENT_H01_ALL, MODEL_PS)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P, STR_PS], 
                     colors=[C_EMPIRICAL, C_H0, C_M1, C_M2],
                     x_labels=SELECTION_EXC_COMPARTMENT_H01_ALL,
                     y_lim=(1, 10**5), use_log=True, error_bars=True,
                     adjust_left=0.1, fig_size=figsize_mm_to_inch(180,50), 
                     filename=plot_folder/f"H01_{SYNCOUNT_EXC_SUBCELLULAR}_all.png")
display(img)

In [ ]:
datasets = [stats.to_numpy(SELECTION_INH_COMPARTMENT_H01_ALL, EMPIRICAL), 
            stats.to_numpy(SELECTION_INH_COMPARTMENT_H01_ALL, MODEL_NULL),
            stats.to_numpy(SELECTION_INH_COMPARTMENT_H01_ALL, MODEL_P),
            stats.to_numpy(SELECTION_INH_COMPARTMENT_H01_ALL, MODEL_PS)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P, STR_PS], 
                     colors=[C_EMPIRICAL, C_H0, C_M1, C_M2],
                     x_labels=SELECTION_INH_COMPARTMENT_H01_ALL,
                     y_lim=(1, 10**5), use_log=True, error_bars=True,
                     adjust_left=0.1, fig_size=figsize_mm_to_inch(180,50), 
                     filename=plot_folder/f"H01_{SYNCOUNT_INH_SUBCELLULAR}_all.png")
display(img)

In [ ]:
datasets = [stats.to_numpy(SELECTION_OTHER_COMPARTMENT_H01_ALL, EMPIRICAL), 
            stats.to_numpy(SELECTION_OTHER_COMPARTMENT_H01_ALL, MODEL_NULL),
            stats.to_numpy(SELECTION_OTHER_COMPARTMENT_H01_ALL, MODEL_P),
            stats.to_numpy(SELECTION_OTHER_COMPARTMENT_H01_ALL, MODEL_PS)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P, STR_PS], 
                     colors=[C_EMPIRICAL, C_H0, C_M1, C_M2],
                     x_labels=SELECTION_OTHER_COMPARTMENT_H01_ALL,
                     y_lim=(1, 10**5), use_log=True, error_bars=True,
                     adjust_left=0.1, fig_size=figsize_mm_to_inch(180,50), 
                     filename=plot_folder/f"H01_synapsecounts_OTHER_compartment_all.png")
display(img)

In [ ]:
datasets = [stats.to_numpy(SELECTION_UNKNOWN_COMPARTMENT_H01_ALL, EMPIRICAL), 
            stats.to_numpy(SELECTION_UNKNOWN_COMPARTMENT_H01_ALL, MODEL_NULL),
            stats.to_numpy(SELECTION_UNKNOWN_COMPARTMENT_H01_ALL, MODEL_P),
            stats.to_numpy(SELECTION_UNKNOWN_COMPARTMENT_H01_ALL, MODEL_PS)]

img = plot_bar_chart(datasets,
                     dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_P, STR_PS], 
                     colors=[C_EMPIRICAL, C_H0, C_M1, C_M2],
                     x_labels=SELECTION_UNKNOWN_COMPARTMENT_H01_ALL,
                     y_lim=(1, 10**8), use_log=True, error_bars=True,
                     adjust_left=0.1, fig_size=figsize_mm_to_inch(180,50), 
                     filename=plot_folder/f"H01_synapsecounts_UNKNOWN_compartment_all.png")
display(img)

In [ ]:
matrix_analyzer = ConnectomeMatrixAnalyzer(df_summary, plot_folder)
matrix_analyzer.set_selection(exclude_filter={})
matrix_analyzer.set_data_columns("model-PS_preference")
matrix_analyzer.build_matrix(["pre_celltype_merged"], 
                             ["post_celltype_merged", "post_compartment"], 
                             row_domains = H01.UNK_EXC_INH_OTHER[1:3],  
                             col_domains = [(H01.UNK_EXC_INH_OTHER[1], *H01.SOMA), (H01.UNK_EXC_INH_OTHER[1], *H01.DEND), (H01.UNK_EXC_INH_OTHER[1], *H01.AIS), \
                                             (H01.UNK_EXC_INH_OTHER[2], *H01.SOMA), (H01.UNK_EXC_INH_OTHER[2], *H01.DEND), (H01.UNK_EXC_INH_OTHER[2], *H01.AIS)], 
                             value_label_map = {
                                 "pre_celltype_merged" : H01.CELLTYPE_LABELS,
                                 "post_celltype_merged" : H01.CELLTYPE_LABELS,
                                 "post_compartment" : H01.COMPARTMENT_LABELS,
                             },
                             aggregation_fn="mean",
                             default_value=1)
matrix_analyzer.render_matrix("H01-model-PS-preference", 
                              colormap_name="coolwarm", vmin=-1, vmax=1, normalization_function=NormalizePreferenceValue(0.5, 4),
                              col_separator_lines=True, row_separator_lines=True)

In [ ]:
matrix_analyzer = ConnectomeMatrixAnalyzer(df_summary, plot_folder)
matrix_analyzer.set_selection(exclude_filter={})
matrix_analyzer.set_data_columns("model-PS_preference")
matrix_analyzer.build_matrix(["pre_celltype_merged"], 
                             ["post_celltype_merged", "post_compartment"], 
                             row_domains = H01.EXC_INH_OTHER_UNK,  
                             col_domains = [(H01.UNK_EXC_INH_OTHER[1], *H01.SOMA), (H01.UNK_EXC_INH_OTHER[1], *H01.DEND), (H01.UNK_EXC_INH_OTHER[1], *H01.AIS), 
                                             (H01.UNK_EXC_INH_OTHER[2], *H01.SOMA), (H01.UNK_EXC_INH_OTHER[2], *H01.DEND), (H01.UNK_EXC_INH_OTHER[2], *H01.AIS),
                                             (H01.UNK_EXC_INH_OTHER[3], *H01.SOMA), (H01.UNK_EXC_INH_OTHER[3], *H01.DEND), (H01.UNK_EXC_INH_OTHER[3], *H01.AIS),
                                             (H01.UNK_EXC_INH_OTHER[0], *H01.SOMA), (H01.UNK_EXC_INH_OTHER[0], *H01.DEND), (H01.UNK_EXC_INH_OTHER[0], *H01.AIS)], 
                             value_label_map = {
                                 "pre_celltype_merged" : H01.CELLTYPE_LABELS,
                                 "post_celltype_merged" : H01.CELLTYPE_LABELS,
                                 "post_compartment" : H01.COMPARTMENT_LABELS,
                             },
                             aggregation_fn="mean",
                             default_value=1)
matrix_analyzer.render_matrix("H01-model-PS-preference_all", 
                              colormap_name="coolwarm", vmin=-1, vmax=1, normalization_function=NormalizePreferenceValue(0.5, 4),
                              col_separator_lines=True, row_separator_lines=True)

#### Synapse clusters

In [ ]:
dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_PS, STR_PSC]

datasets = [stats.to_numpy(SELECTION_CLUSTER_6, EMPIRICAL), 
            stats.to_numpy(SELECTION_CLUSTER_6, MODEL_NULL),
            stats.to_numpy(SELECTION_CLUSTER_6, MODEL_PS),
            stats.to_numpy(SELECTION_CLUSTER_6, MODEL_PSCb)[-4000:,:]]

labels = [l.split("-")[1] for l in SELECTION_CLUSTER_6]
        
filename = plot_folder/f"H01_{MODEL_PSC}_{SYNCLUSTERS}.png"
img = plot_line_chart(datasets, x_labels=labels, linestyles=[".-","x-","x-","x-"], 
                      capsize=2, linewidth=1, marker_size=4,  
                      colors=[C_EMPIRICAL, C_H0, C_M1, C_M2], y_lim=(-0.1, 10**5), x_lim=(-0.25,6.25),       
    dataset_labels=None, error_bars=True, use_log=True, hline_y=None,  adjust_left=0.15, adjust_bottom=0.15, fig_size=figsize_mm_to_inch(70,60), filename=filename)
display(img)

#### Triplet motifs

In [ ]:
dataset_labels = [STR_EMPIRICAL, STR_NULL, STR_PS, STR_PSCb]

print(stats.to_numpy(SELECTION_MOTIF, MODEL_NULL).shape)

datasets = [stats.to_numpy(SELECTION_MOTIF, EMPIRICAL), 
            stats.to_numpy(SELECTION_MOTIF, MODEL_NULL),
            np.atleast_2d(stats.to_numpy(SELECTION_MOTIF, MODEL_PS)),
            np.atleast_2d(stats.to_numpy(SELECTION_MOTIF, MODEL_PSCb))]

filename = plot_folder/f"H01_{MOTIFS}.png"
img = plot_motifs_bar_chart(datasets, 
    colors = [C_EMPIRICAL, C_H0, C_M1, C_M2],
    selected_motifs=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16),
    dataset_labels=dataset_labels, quantile_low=25, quantile_high=75,
    use_mean=False, marker_size=4, capsize=0, y_axis_label="occurrences relative to random",
    error_bars=True, y_lim=(-0.5,10**3), use_log=True, fig_size=figsize_mm_to_inch(160,50), adjust_left=0.07, filename=filename)
display(img)